# TILT - Teemo Induced Loss of Tranquility
A research project to study main factors in inducing tilt.

ONLY LOOKS AT THE PERSON STATS THEMSELVES


# Imports

In [1]:
# from collections import defaultdict
from pathlib2 import Path
from riotwatcher import LolWatcher, ApiError

# import arrow
# import csv
import itertools
# import datetime
# import time
import json
# import numpy as np
import os
import pandas as pd
import roleml

# Functions

In [2]:
  
def api_key(api_key_loc):
    """
    Read in the development API key from a file and checks if it is viable.
    
    Args:
        credentials (str): Name of json file containing the credentials.
    Returns:
        api_key (str): The API key.
    """

    while True:
        with open(api_key_loc, "r") as json_data:
            creds = json.load(json_data)
            api_key = creds["dev_api_key"]
            lol_watcher = LolWatcher(creds["dev_api_key"])
            try:
                # Validate API key by using it to check server status
                lol_watcher.lol_status.shard_data("euw1")
                # Break if key is functional
                break
            except ApiError as error:
                # If the current API key does not work input new one
                if error.response.status_code == 403:
                    new_api_key = input("API key is incorrect, enter correct key here.")
                    creds["dev_api_key"] = new_api_key
                    # Replace the old API key
                    with open(api_key_loc, "w") as json_data:
                        json.dump(creds, json_data)
    return api_key

In [3]:
def trans_reg(reg_abbrv):
    """
    Translate a league of legends region into a region readable by riot watcher.
    
    Args:
        reg_abbrv (str): Abbreviation of a official registered Riot servers that
                         hosts league of legends (e.g. euw1).
    Returns:
        rw_region (str): Riot Watcher region the reg_abbrv server falls under.
    """
    # Look up in the list what the riot watcher region is for the given region abbreviation
    regions_metadata = {"br1": "americas",
                        "eun1": "europe",
                        "euw1": "europe",
                        "jp1": "asia",
                        "kr": "asia",
                        "la1": "americas",
                        "la2": "americas",
                        "na1": "americas",
                        "oc1": "americas",
                        "tr1": "europe",
                        "ru": "europe"
                        }

    for reg, rw_reg in regions_metadata.items():
        if reg_abbrv.lower() == reg:
            rw_region = rw_reg
    
    return rw_region

In [4]:
def get_matches(reg, pid, s_time, e_time):
    """
    Retrieve match IDs from a summoner within a given timeframe.

    Args:
        reg (str): Abbreviation of a official registered Riot servers that
                   hosts league of legends (e.g. euw1).
        pid (str): An Encrypted globally unique identifyer for a summoner.
        s_time (long): Start of a timeframe in (milli)seconds following
                       the Coordinated Universal Time (UTC) format.
        e_time (long): end of a timeframe in (milli)seconds following
                       the Coordinated Universal Time (UTC) format.
    Returns:
        matches (list): League of legends match IDs in chronological order
                        starting with the most recent match.
    """

    # Retrieve all match ID's between two time points 
    matches = lol_watcher.match.matchlist_by_puuid(region=trans_reg(reg),
                                                   puuid=pid,
                                                   start_time=s_time,
                                                   end_time=e_time,
                                                   count=100)
    
    # Geather more match IDs in case matches exceeds the standard limit of 100
    while len(matches) % 100 == 0 and len(matches) != 0:
        # Geather match details of earliest match
        match_deets = lol_watcher.match.by_id(trans_reg(reg), matches[-1])
        # Start time of earliest match
        early_g_start = int(str(match_deets["info"]['gameCreation'])[:10])
        # Select 100 matches previous to early_g_start  
        match_add = lol_watcher.match.matchlist_by_puuid(region=trans_reg(reg),
                                                         puuid=pid,
                                                         start_time=s_time,
                                                         end_time=early_g_start,
                                                         count=100)
        if len(match_add) == 0:
            break
        else:
            matches.extend(match_add)

    return matches

In [19]:
def geather_data(puuid, matches):
    """
    Geather summoner data from a match and chronology data of the match.
    
    Args:
        puuid (str): An Encrypted globally unique identifyer for a summoner.
        matches (list): League of legends match IDs in chronological order
                        starting with the most recent matches.  
    Returns:
        df_matches_data (df): Data geathered from matches of a summoner.
    """
    
    time_info = ["gameCreation", "gameStartTimestamp", "gameDuration"]
    keep_data = ["puuid", "kills", "assists", "deaths", 
                 "doubleKills", "tripleKills", "quadraKills", "pentaKills", 
                 "killingSprees", "largestKillingSpree", 
                 "teamEarlySurrendered", "gameEndedInEarlySurrender", "gameEndedInSurrender",
                 "neutralMinionsKilled", "totalMinionsKilled", "teamId", "win"]

    col_names = ["match_id", "puuid", "kills", "assists", "deaths", 
                 "2_kills", "3_kills", "4_kills", "5_kills", 
                 "kill_spree", "max_kill_spree", 
                 "early_surr_try", "early_surr", "game_surr",
                 "neutral_kills", "mini_kills", "team_id", "win",
                 "game_make", "game_start", "game_dur"]
      
    comp_data = []
    # Geather match details
    for match_id in matches:
        reg = match_id.split("_")[0].lower()
        match_deets = lol_watcher.match.by_id(trans_reg(reg), match_id)        
        # Collect time data
        time_data = dict((key, match_deets["info"][key]) for key in time_info)

        # Collect summoner data       
        for part_info in match_deets["info"]["participants"]:
            if part_info["puuid"] == puuid:
                filt_data = dict((key, part_info[key]) for key in keep_data)
        match_data = {"match_id": match_id} | filt_data | time_data
        comp_data.append(match_data.values())
    
    # Store all data in a dataframe
    df_matches_data = pd.DataFrame(comp_data, columns=col_names)

    return df_matches_data
    

In [16]:
def filt_matches(matches_data, max_rest, min_streak):
    """
    Filter matches based on rest time in between matches and number of matches played in a row.  
    
    Args:
        matches_data (df): Data geathered from matches of a summoner.
        max_rest (int): Rest time in between matches in miliseconds e.g. 3600000 = 1 hour.
        min_streak (int): Minimum games played in a row with less then max_rest between.
    Returns:
        df_match_filt (df): Matches filtered based on minuum subsequent matches within
                            the maximum rest time in between the matches.
    """
      
    # Filter for matches based on rest time

    # Add rest time
    df_match_filt = matches_data 
    df_match_filt["game_end"] = df_match_filt.loc[:,["game_start", "game_dur"]].sum(axis=1)
    df_match_filt["prev_game_make"] = df_match_filt["game_make"].shift(-1, fill_value=0)
    df_match_filt["rest_time"] = df_match_filt["game_end"] - df_match_filt["prev_game_make"]


    # Store if the game ID should be kept if rest time is below max_rest
    df_match_filt['keep'] = df_match_filt["rest_time"] <= max_rest
    # Grouping based on matches played subsequently 
    subseq_true = df_match_filt['keep'].diff().ne(0).cumsum()
    # Drop streaks that are lower then min_streak
    df_match_filt = df_match_filt[df_match_filt['keep'].groupby(subseq_true).transform("count") >= min_streak]
    # Drop matches with rest times above max_rest 
    df_match_filt = df_match_filt[~(df_match_filt["keep"] == 0.0)]

    # Add the streak id to the df_match_filt
    df_match_filt["streak_id"] = df_match_filt.groupby(subseq_true).grouper.group_info[0]

    # Remove the keep column 
    df_match_filt.drop("keep", axis=1, inplace = True)
    
    return df_match_filt


In [26]:
reg="EUW1"

In [37]:
def pred_role(df_match_filt):
    """
    Filter matches based on rest time in between matches and number of matches played in a row.  
    
    Args:
        df_match_filt (df): Matches filtered based on minuum subsequent matches within
                            the maximum rest time in between the matches.
    Returns:
        df_match (df): df_match_filt with added in game role of player.
    """
    
    roles=[]
    for match_id, puuid in zip(df_match_filt['match_id'], df_match_filt['puuid']):
        match = lol_watcher.match.by_id(trans_reg(reg), match_id)
        match_deets = match["metadata"] | match["info"]
        match_time_line = lol_watcher.match.timeline_by_match(trans_reg(reg), match_id)
        # print(match_deets)
        print(roleml.predict(match_deets, match_time_line))  
        # print(match_id, puuid)
    


    return None

In [38]:
# result = [pred_role(match_id, puuid) for match_id, puuid in zip(df_match_filt['match_id'], df_match_filt['puuid'])]
df_match_filt=geather_data("PkyDyFsJQ35GQYruyFBfALnEvWKhRNSPqNyq15L6wgmY4WWH_ddk1ufUMtNh2-ioNtJvHwo-aY3c-w",['EUW1_5395088971'])

pred_role(df_match_filt)

IncorrectMap: This package only handles Summoner’s Rift games.

In [8]:

def get_summoner_data(regs, tiers, divs, sum_lim, s_time, e_time, max_rest, min_streak, sumo_data_out):
    """
    Get puuid's, tier and division, from all regions that are in ranks below master rank ((random?) above bronze?).
    
    Args:
        regs (list): Official registered Riot servers that hosts league of legends.
        tier (list): Tiers below Master rank.
        divs (list): Divisions in roman numerals.
        sum_lim (int): Maximum number of summoner ids to collect per region, tier and division
        s_time (long): Start of a timeframe in (milli)seconds following
                       the Coordinated Universal Time (UTC) format.
        e_time (long): end of a timeframe in (milli)seconds following
                       the Coordinated Universal Time (UTC) format.
        max_rest (int): Rest time in between matches in miliseconds e.g. 3600000 = 1 hour.
        min_streak (int): Minimum games played in a row with less then max_rest between.
        sumo_data_out (str): location to store the info on the summoner and game statistics.
    Returns:
        None
    """
    
    # for reg, tier, div in itertools.product(regs, tiers, divs): # real line change what we want in the settings file
    for reg, tier, div in itertools.product(regs, tiers, divs[0:1]): # test line
        # print(reg, tier, div) # TODO: remove me later
        summs_div = 0
        page_nr = 0
        # Keep adding new summoners until the summoner limit has been reached
        while summs_div < sum_lim:
            page_nr += 1
            summs = lol_watcher.league.entries(reg, "RANKED_SOLO_5x5", tier, div, page_nr)
            # Look into data per summoner
            for sumo in summs:
                if summs_div < sum_lim:
                    # Get PUUID
                    pid = lol_watcher.summoner.by_id(reg, sumo["summonerId"])["puuid"]

                    # Retrieve all match ID's between two time points
                    match_ids = get_matches(reg, pid, s_time, e_time)
                    # Skip summoners that have no matches in the given time frame 
                    if not match_ids:
                        continue
        
                    # Geather match data of the summoner
                    match_info = geather_data(pid, match_ids)

                    # Filter matches
                    filt_match_info = filt_matches(match_info, max_rest, min_streak)
                    # Skip summoners that have no matches left after filtering
                    if filt_match_info.empty:
                        continue
                    
                    
                    # Add summoner rank information 
                    cur_rank = f"{tier}_{div}"
                    filt_match_info.insert (2, "rank", cur_rank)

                    # Increase the summoner counter
                    summs_div += 1
                    
                    # Store summoner data in a tsv file
                    file_true = sumo_data_out.exists()
                    filt_match_info.to_csv(sumo_data_out,
                                           header=not file_true,
                                           mode='a' if file_true else 'w',
                                           sep="\t")

# Global variables and settings

In [9]:
# Directory locations

# Project folders
proj_dir = Path.cwd().parent

# Raw data storage
data_dir = proj_dir / "data"

# Out dir
out_dir = proj_dir / "out"

# Gobal variables

# Set API key
api_key_loc = data_dir / "dev_api_key.json"

# Enter API key
lol_watcher = LolWatcher(api_key(api_key_loc))

# Read user settings 
settings_loc = proj_dir / "settings" / "config.json"
with open(settings_loc, "r") as settings_data:
    settings = json.load(settings_data)


### Data collection

In [52]:
# Retrieve the summoner info of n summoners

# Remove output file if it exists
out_file_path = out_dir / "summoner_data.tsv"
if out_file_path.exists():
    os.remove(out_file_path)

get_summoner_data(regs=settings["regions"],
                  tiers=settings["tiers"],
                  divs=settings['divisions'],
                  sum_lim=settings['summoner_limit'],
                  s_time=settings["start_time"],
                  e_time=settings["end_time"],
                  max_rest=settings["max_rest"],
                  min_streak=settings["min_streak"],
                  sumo_data_out=out_dir / "summoner_data.tsv")




# # Read in the summoner data
# sumo_info = pd.read_csv(summs_data_out, sep = "\t")


# print(summs_info)

summs_div=1, pagenr=1
summs_div=2, pagenr=1


In [ ]:
Use timeline data
Finally, we can use most-match data in the timeline objects to identify the position on the map of the champions
throughout the game. We can also look at items, runes, summoner spells, etc. to help narrow down what position
the champion likely played in. This is easiest implemented using a machine learning approach. Training data can be
found here: https://github.com/Canisback/roleML/blob/master/data/verification_results.csv. An implementation of an
SVM (a machine learning model) to identify lanes and roles can be found here: https://github.com/meraki-analytics/
role-identification/blob/timeline/timeline-roleidentification/finished_timeline_roleID.ipynb that you can use as an example for your own model. A decision tree is another good machine learning algorithm for this task.


### Data Analysis

In [ ]:
# Base line
# base line = all games of a player
# tilt is calculated as win % er streak increase or decrease
# % win on first game
# % win on secoond game
# % win on third game etc

# make 2 datasets games long rest, games short rest 
# long rest >
# short rest <=
# 
#





#### Importance of rest

In [ ]:
# REst time vs win rate plot
# 
# check if player has games with short rest take all games fitler is done later in data analysis
# 
#  

#### Player profiles

In [ ]:
# WHen to players play? 
# player to time point


#### Tilt or in the zone?

In [ ]:
# winrate increase or decrease by factor of previous game
# base winrate?
# define good and bad stats using medians/average normal dist? 
# Win rate in next game after losing with bad stats
# Win rate in next game after losing with good stats
# Win rate in next game after winning with bad stats
# Win rate in next game after winning with good stats
